# Overview Materi

Source: https://www.youtube.com/watch?v=LDRbO9a6XPU

Jelaskan secara singkat apa itu decision tree menurut pemahamanmu!

Decision tree adalah sebuah algoritma yang digunakan untuk memprediksi hasil dengan cara membagi data menjadi bercabang seperti pohon, hingga dalam satu node hanya memuat 1 label sehingga mudah untuk dipahami.


# Import Data & Libraries

In [2]:
from __future__ import print_function

# label kolom
header = ["color", "diameter", "label"]

# data training
training_data = [
    ['Green', 3, 'Apple'],
    ['Yellow', 3, 'Apple'],
    ['Red', 1, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon'],]

# data testing
testing_data = [
    ['Green', 3, 'Apple'],
    ['Yellow', 4, 'Apple'],
    ['Red', 2, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon'],]

# Fungsi Dasar

In [4]:
# fungsi mencari apa saja unique value dari suatu kolom
def unique_value(rows, col):
    return set([row[col]for row in rows])

# contoh penggunaan:

# mencari unique value dari kolom "color"
unique_colors = unique_value(training_data, 0)
print(unique_colors)

# mencari unique value dari kolom "diameter"
unique_diameters = unique_value(training_data, 1)
print(unique_diameters)

{'Yellow', 'Green', 'Red'}
{1, 3}


In [5]:
# fungsi Menghitung jumlah unique value dari suatu kolom
def class_counts(rows):
    counts = {}
    for row in rows:
        label = row[-1]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

# contoh penggunaan
counts = class_counts(training_data)
print(counts)

{'Apple': 2, 'Grape': 2, 'Lemon': 1}


In [6]:
# fungsi pengecekan suatu value numerik atau bukan
def is_numeric(value):
    return isinstance(value, int) or isinstance(value, float)

# contoh penggunaan
print(is_numeric(3))
print(is_numeric("Red"))

True
False


In [9]:
# kelas untuk merepresentasikan pertanyaan pada decision tree
class Question:

    # inisialisasi kolom dan nilai pertanyaan
    def __init__(self, column, value):
        self.column = column
        self.value = value

    # mengecek apakah contoh data sesuai dengan pertanyaan
    def match(self, example):
        val = example[self.column]
        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value

    # menampilkan pertanyaan dalam format string yang mudah dibaca
    def __repr__(self):
        condition = "=="
        if is_numeric(self.value):
            condition = ">="
        return "Is %s %s %s?" % (
            header[self.column], condition, str(self.value))

# contoh penggunaan 1
question = Question(0, 'Green')
print(question)
print(question.match(training_data[0]))

# contoh penggunaan 2
question = Question(1, 3)
print(question)
print(question.match(training_data[0]))

Is color == Green?
True
Is diameter >= 3?
True


In [8]:
# membagi dataset menjadi dua berdasarkan pertanyaan
def partition(rows, question):
    true_rows, false_rows = [], []
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows

# contoh penggunaan
true_rows, false_rows = partition(training_data, Question(0, 'Red'))
print(true_rows)
print(false_rows)

[['Red', 1, 'Grape'], ['Red', 1, 'Grape']]
[['Green', 3, 'Apple'], ['Yellow', 3, 'Apple'], ['Yellow', 3, 'Lemon']]


**apa itu gini impurity?**
<br> gini impurity berfungsi mengukur tingkat ketidakmurnian atau ketidakteraturan pada sebuah simpul (node) dalam pohon

In [11]:
# menghitung nilai Gini Impurity untuk sebuah dataset
def gini(rows):
    counts = class_counts(rows)
    impurity = 1
    for lbl in counts:
        prob_of_lbl = counts[lbl] / float(len(rows))
        impurity -= prob_of_lbl**2
    return impurity

# contoh penggunaan
print(gini(training_data))

0.6399999999999999


**apa itu information gain?**
<br> information gain berfungsi mengukur seberapa efektif sebuah fitur dalam memisahkan data berdasarkan kelas-kelasnya

In [12]:
# menghitung nilai Information Gain dari pemisahan dataset
def info_gain(left, right, current_uncertainty):
    p = float(len(left)) / (len(left) + len(right))
    return current_uncertainty - p * gini(left) - (1 - p) * gini(right)

# contoh penggunaan
true_rows, false_rows = partition(training_data, Question(0, 'Red'))
print(info_gain(true_rows, false_rows, gini(training_data)))

0.37333333333333324


In [18]:
# mencari pertanyaan terbaik untuk membagi dataset berdasarkan information gain tertinggi
def find_best_split(rows):
    best_gain = 0  # Keep track of the best information gain
    best_question = None  # Keep track of the question that produced the best gain
    current_uncertainty = gini(rows)
    n_features = len(rows[0]) - 1  # number of columns

    for col in range(n_features):  # for each column
        values = unique_value(rows, col)  # unique values in the column

        for val in values:  # for each value
            question = Question(col, val)

            # splitting the dataset
            true_rows, false_rows = partition(rows, question)

            # Skip this split if it doesn't divide the dataset
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue

            # Calculate the information gain from this split
            gain = info_gain(true_rows, false_rows, current_uncertainty)
            if gain >= best_gain:
                best_gain, best_question = gain, question

    return best_gain, best_question

# contoh penggunaan
best_gain, best_question = find_best_split(training_data)
print(best_gain)
print(best_question)

0.37333333333333324
Is diameter >= 3?


# Fungsi Decision Tree

In [ ]:
# merepresentasikan node daun (leaf) pada decision tree yang berisi hasil prediksi
class Leaf:

    # inisialisasi leaf dengan menghitung jumlah kemunculan tiap kelas
    def __init__(self, rows):
        ...

In [ ]:
# merepresentasikan node keputusan (decision node) yang berisi pertanyaan dan cabang
class Decision_Node:

    # inisialisasi node dengan pertanyaan, cabang benar, dan cabang salah
    def __init__(self,
                 question,
                 true_branch,
                 false_branch):
        ...


In [ ]:
# membangun decision tree secara rekursif
def build_tree(rows):
    ...
    return Decision_Node(question, true_branch, false_branch)

In [ ]:
# mencetak struktur decision tree secara rekursif dalam format teks
def print_tree(node, spacing=""):

    # base case: jika sudah mencapai leaf
    ...

    # mencetak pertanyaan pada node saat ini
    ...

    # mencetak cabang true secara rekursif
    ...

    # mencetak cabang false secara rekursif
    ...

# contoh penggunaan
...

In [ ]:
# mengklasifikasikan satu baris data menggunakan decision tree
def classify(row, node):

    # base case: jika sudah mencapai leaf
    ...

    # menentukan apakah mengikuti cabang true atau cabang false
    # dengan membandingkan nilai fitur pada baris dengan pertanyaan di node
    ...

# contoh penggunaan
...

In [ ]:
# menampilkan prediksi pada leaf dalam format persentase
def print_leaf(counts):
    ...
    return probs

# contoh penggunaan
...

# Predict Using Decision Tree

In [ ]:
# menguji decision tree dengan data uji dan membandingkan hasil prediksi dengan label asli
for row in testing_data:
    ...